In [1]:
import os
import sys

import torch


import matplotlib.pyplot as plt
import numpy as np
from gpytorch.kernels.kernel import dist

# import torch
from gpytorch.kernels.rbf_kernel import postprocess_rbf

# from bench_functions import micha, rose
# from botorch_gp import BotorchGp
from sklearn.metrics import mean_squared_error
model_directory = "C:/Projekte/DGCN/covnet2"
sys.path.append(model_directory)

# Now you can import Model from model.py
from model import Model
from config import configs


In [2]:
import pandas as pd
X = pd.read_csv("C:/Projekte/phm 2024/Data/X_train.csv")
y = pd.read_csv("C:/Projekte/phm 2024/Data/y_train.csv")

In [3]:
X.head()

,id,trq_measured,oat,mgt,pa,ias,np,ng
0,0,54.100,2.00000,544.5000,212.1408,74.56250,89.18000,99.6400
1,1,49.625,24.22231,578.4844,1625.6400,30.35596,99.55273,91.3866
2,2,52.000,7.00000,566.1000,1912.9250,65.62500,100.14000,90.9600
3,3,62.400,7.25000,560.1000,277.0632,54.81250,90.64000,100.2800
4,4,62.900,23.25000,593.7000,53.6448,73.43750,99.91000,92.1700


In [4]:
y.head()

,id,faulty,trq_margin
0,0,1,-13.717745
1,1,0,1.791863
2,2,1,-13.944871
3,3,0,-0.017281
4,4,0,7.322404


In [35]:
ids = X["id"]
X_train = X.drop("id", axis=1)

y_trq = y["trq_margin"]
y_faulty = y["faulty"]

In [6]:
len(y_trq)

742625

In [26]:
X_np = np.array(X_train[:10000])
y_trq_np = np.array(y_trq[:10000])
y_faulty_np = np.array(y_faulty[:10000])

In [17]:
len(X_np)


10000

In [18]:
model = Model(
    X_np, y_trq_np, noise=False, batch_size=512, neurons=50, learning_rate=1e-2
)
model.fit(epochs=150, verbose=True)

Epoch:   0%|          | 0/150 [00:00<?, ?it/s]

c:\Users\je009447\AppData\Local\miniconda3\envs\bayesnewton\lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


In [19]:
y_pred = model.predict(np.array(X_train[10000:15000]))

In [20]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_pred, np.array(y_trq[10000:15000]))

In [21]:
mse

0.11290295152679089

In [23]:
from sklearn.linear_model import LinearRegression
lin = LinearRegression()
lin.fit(X_np, y_trq_np)
y_pred = lin.predict(np.array(X_train[10000:15000]))
mse = mean_squared_error(y_pred, np.array(y_trq[10000:15000]))
mse

5.320451391219264

In [27]:
class_model = Model(
    X_np, y_faulty_np, noise=False, batch_size=512, neurons=50, learning_rate=1e-2
)
class_model.fit(epochs=150, verbose=True)

c:\Users\je009447\AppData\Local\miniconda3\envs\bayesnewton\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch:   0%|          | 0/150 [00:00<?, ?it/s]

c:\Users\je009447\AppData\Local\miniconda3\envs\bayesnewton\lib\site-packages\gpytorch\likelihoods\gaussian_likelihood.py:300: GPInputWarning: You have passed data through a FixedNoiseGaussianLikelihood that did not match the size of the fixed noise, *and* you did not specify noise. This is treated as a no-op.
  warnings.warn(
c:\Users\je009447\AppData\Local\miniconda3\envs\bayesnewton\lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


In [28]:
y_pred = model.predict(np.array(X_train[10000:15000]))
y_pred = np.round(y_pred)

In [ ]:
y_pred, lcb, ucb = model.predict(np.array(X_train[10000:15000]), pred_var=True)

In [30]:
y_pred

tensor([[-7.],
        [-6.],
        [-4.],
        ...,
        [ 8.],
        [14.],
        [-4.]], dtype=torch.float64)

In [29]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_pred, np.array(y_faulty[10000:15000]))
acc

0.026

In [32]:
from xgboost import XGBRegressor
xgboost = XGBRegressor()
xgboost.fit(X_np, y_trq_np)
y_pred_xg = xgboost.predict(np.array(X_train[10000:15000]))
mse = mean_squared_error(y_pred, np.array(y_trq[10000:15000]))
print(mse)

0.1910663424724587


In [34]:
from xgboost import XGBClassifier

xgboost = XGBClassifier()
xgboost.fit(X_np, y_faulty_np)
y_pred_xg = xgboost.predict(np.array(X_train[10000:15000]))
acc = accuracy_score(y_pred_xg, np.array(y_faulty[10000:15000]))
print(acc)

0.9832


In [ ]:
lin = LinearRegression()
lin.fit(X_np, y_trq_np)
y_pred = lin.predict(np.array(X_train[10000:15000]))
mse = mean_squared_error(y_pred, np.array(y_trq[10000:15000]))
mse

In [ ]:
error_margin = upper_bound - y_pred_reg
se = error_margin/1.96
# se is the standard deviation

In [ ]:
output_data = {}
for i in ids:
    {
        "class": y_pred_class[i - 1],
        "class_conf": y_pred_class_conf[i - 1],
        "pdf_type": "norm",
        "pdf_args": {"loc": y_pred_reg, "scale": y_pred_reg_dev},
    }
    output_data[i] = {}

In [ ]:
import json
with open("output.json", "w") as json_file:
    json.dump(output_data, json_file, indent=4)